In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [2]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [3]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [4]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

--2018-07-19 14:16:55--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 52.219.72.6
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|52.219.72.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘flights14.csv.8’

flights14.csv.8     100%[===================>]  15.40M  7.52MB/s    in 2.0s    

2018-07-19 14:16:58 (7.52 MB/s) - ‘flights14.csv.8’ saved [16150465/16150465]



In [5]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [6]:
flightsDF = sqlContext.createDataFrame(flights)

In [7]:
flightsDF.count()

253316

In [8]:
flightsDF.show()

+--------+---------+--------+---------+-------+---+---------+--------+----+--------+------+----+---+-----+------+-------+----+
|air_time|arr_delay|arr_time|cancelled|carrier|day|dep_delay|dep_time|dest|distance|flight|hour|min|month|origin|tailnum|year|
+--------+---------+--------+---------+-------+---+---------+--------+----+--------+------+----+---+-----+------+-------+----+
|     359|       13|    1238|        0|     AA|  1|       14|     914| LAX|    2475|     1|   9| 14|    1|   JFK| N338AA|2014|
|     363|       13|    1523|        0|     AA|  1|       -3|    1157| LAX|    2475|     3|  11| 57|    1|   JFK| N335AA|2014|
|     351|        9|    2224|        0|     AA|  1|        2|    1902| LAX|    2475|    21|  19|  2|    1|   JFK| N327AA|2014|
|     157|      -26|    1014|        0|     AA|  1|       -8|     722| PBI|    1035|    29|   7| 22|    1|   LGA| N3EHAA|2014|
|     350|        1|    1706|        0|     AA|  1|        2|    1347| LAX|    2475|   117|  13| 47|    1|   JF

In [9]:
flightsDF.createOrReplaceTempView("flight")

In [10]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [11]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [12]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

### Find the flightwith the longest arrival delay

### Find the top 10 destinations ordered by the number of flights

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

### For all flights from JFK during June, show the average departure delay for each destination

### For every origin/dest pair, count the number of flights

In [13]:
sqlContext.sql("select * from flight where arr_delay > 0 and dep_delay < 0").count()

30239

In [14]:
flightsDF.where('arr_delay > 0 and dep_delay < 0 ').count()

30239

In [15]:
sqlContext.sql("select * from flight order by arr_delay desc").limit(1).take(1)

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)]

In [16]:
flightsDF.groupby(['dest'])\
.agg({"*": "COUNT"})\
.sort("count(1)", ascending=False)\
.show(10)

+----+--------+
|dest|count(1)|
+----+--------+
| LAX|   14434|
| ATL|   12808|
| SFO|   11907|
| MCO|   11709|
| BOS|   11609|
| ORD|   11589|
| MIA|    9928|
| CLT|    9624|
| FLL|    9471|
| DCA|    6748|
+----+--------+
only showing top 10 rows



In [17]:
sqlContext.sql("select dest, count(*) as count from flight group by dest order by count desc").show(10)

+----+-----+
|dest|count|
+----+-----+
| LAX|14434|
| ATL|12808|
| SFO|11907|
| MCO|11709|
| BOS|11609|
| ORD|11589|
| MIA| 9928|
| CLT| 9624|
| FLL| 9471|
| DCA| 6748|
+----+-----+
only showing top 10 rows



In [18]:
flightsDF\
.filter(flightsDF.arr_delay > 0)\
.groupby(['dest'])\
.agg({"arr_delay":"AVG"})\
.sort("avg(arr_delay)", ascending=False)\
.show(10)    

+----+------------------+
|dest|    avg(arr_delay)|
+----+------------------+
| EGE| 80.59016393442623|
| AVP|              67.0|
| JAC|59.666666666666664|
| CAK|57.907042253521126|
| TUL| 56.99152542372882|
| MSN| 54.80237154150198|
| IAD|53.324599708879184|
| BGR| 52.73504273504273|
| OKC| 52.14503816793893|
| TVC| 51.15151515151515|
+----+------------------+
only showing top 10 rows



In [19]:
sqlContext.sql("select dest, mean(arr_delay) as avg_delay from flight where arr_delay >0 group by dest order by avg_delay desc").show(10)

+----+------------------+
|dest|         avg_delay|
+----+------------------+
| EGE| 80.59016393442623|
| AVP|              67.0|
| JAC|59.666666666666664|
| CAK|57.907042253521126|
| TUL| 56.99152542372882|
| MSN| 54.80237154150198|
| IAD|53.324599708879184|
| BGR| 52.73504273504273|
| OKC| 52.14503816793893|
| TVC| 51.15151515151515|
+----+------------------+
only showing top 10 rows



In [20]:
flightsDF.sample(withReplacement=False,fraction=0.1,seed=12).agg({"dep_delay": "AVG"}).show()

+------------------+
|    avg(dep_delay)|
+------------------+
|12.518043400071933|
+------------------+



In [21]:
flightsDF\
.filter(flightsDF.origin == "JFK")\
.filter(flightsDF.month == 6)\
.groupby(flightsDF.dest)\
.agg({"dep_delay": "AVG"})\
.show(10)

+----+-------------------+
|dest|     avg(dep_delay)|
+----+-------------------+
| PSE| 17.133333333333333|
| MSY|  10.96629213483146|
| BUR| 16.428571428571427|
| OAK|  22.75862068965517|
| DCA|  5.372781065088757|
| ORF| 10.678571428571429|
| SAV|  4.267857142857143|
| CMH| 11.862068965517242|
| HNL|-1.7586206896551724|
| SJC| 12.241379310344827|
+----+-------------------+
only showing top 10 rows



In [22]:
sqlContext.sql("select distinct(origin) from flight").show()

+------+
|origin|
+------+
|   LGA|
|   EWR|
|   JFK|
+------+



In [23]:
sqlContext.sql("select distinct(dest) from flight").show()

+----+
|dest|
+----+
| PSE|
| MSY|
| BUR|
| SNA|
| GRR|
| PVD|
| GSO|
| MYR|
| OAK|
| MSN|
| DCA|
| ORF|
| SAV|
| CMH|
| CAK|
| CHO|
| IAH|
| HNL|
| SJC|
| CVG|
+----+
only showing top 20 rows



In [24]:
sqlContext.sql("select distinct(origin,dest) from flight").count()

221

In [25]:
# For every origin/dest pair, count the number of flights
sqlContext.sql("select origin, dest, count(*) as count from flight group by origin, dest order by count desc").show()


+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|10208|
|   JFK| SFO| 7368|
|   LGA| ORD| 7052|
|   LGA| ATL| 6925|
|   LGA| MIA| 5084|
|   EWR| SFO| 4539|
|   JFK| MCO| 4467|
|   EWR| BOS| 4268|
|   EWR| LAX| 4226|
|   EWR| ATL| 4182|
|   EWR| MCO| 4164|
|   JFK| BOS| 4111|
|   JFK| SJU| 4027|
|   EWR| CLT| 3921|
|   LGA| DFW| 3789|
|   LGA| DCA| 3753|
|   LGA| DTW| 3663|
|   LGA| CLT| 3431|
|   JFK| LAS| 3355|
|   LGA| FLL| 3304|
+------+----+-----+
only showing top 20 rows



In [26]:
sc.stop()